In [51]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model #for lasso

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) 

Pandas   1.1.3
Sklearn  0.23.2


In [2]:
df = pd.read_csv('Building_Permits.csv')
df.head()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


In [3]:
df.isnull().sum() * 100 / len(df)

Permit Number                              0.000000
Permit Type                                0.000000
Permit Type Definition                     0.000000
Permit Creation Date                       0.000000
Block                                      0.000000
Lot                                        0.000000
Street Number                              0.000000
Street Number Suffix                      98.885872
Street Name                                0.000000
Street Suffix                              1.391654
Unit                                      85.178984
Unit Suffix                               99.014077
Description                                0.145802
Current Status                             0.000000
Current Status Date                        0.000000
Filed Date                                 0.000000
Issued Date                                7.511312
Completed Date                            51.135747
First Construction Document Date           7.514329
Structural N

In [5]:
df.columns

Index(['Permit Number', 'Permit Type', 'Permit Type Definition',
       'Permit Creation Date', 'Block', 'Lot', 'Street Number',
       'Street Number Suffix', 'Street Name', 'Street Suffix', 'Unit',
       'Unit Suffix', 'Description', 'Current Status', 'Current Status Date',
       'Filed Date', 'Issued Date', 'Completed Date',
       'First Construction Document Date', 'Structural Notification',
       'Number of Existing Stories', 'Number of Proposed Stories',
       'Voluntary Soft-Story Retrofit', 'Fire Only Permit',
       'Permit Expiration Date', 'Estimated Cost', 'Revised Cost',
       'Existing Use', 'Existing Units', 'Proposed Use', 'Proposed Units',
       'Plansets', 'TIDF Compliance', 'Existing Construction Type',
       'Existing Construction Type Description', 'Proposed Construction Type',
       'Proposed Construction Type Description', 'Site Permit',
       'Supervisor District', 'Neighborhoods - Analysis Boundaries', 'Zipcode',
       'Location', 'Record ID'],
     

In [6]:
df['Street Suffix'].unique()

array(['St', 'Av', 'Tr', 'Ct', 'Bl', 'Wy', 'Dr', nan, 'Rd', 'Cr', 'Pl',
       'Ln', 'Hy', 'Pk', 'Al', 'Pz', 'Wk', 'Rw', 'So', 'Sw', 'No', 'Hl'],
      dtype=object)

In [17]:
group_cols = df.loc[:, ['Street Number Suffix', 'Zipcode']]
group_cols.head(10)

,Street Number Suffix,Zipcode
0,NaN,94102.0
1,NaN,94102.0
2,NaN,94109.0
3,NaN,94109.0
4,NaN,94102.0
5,NaN,94107.0
6,NaN,94122.0
7,NaN,94124.0
8,NaN,94117.0
9,NaN,94117.0


In [19]:
group_cols.dropna().head(10)

,Street Number Suffix,Zipcode
92,A,94103.0
146,C,94117.0
164,A,94117.0
200,C,94117.0
273,A,94103.0
470,V,94124.0
639,A,94123.0
649,C,94115.0
767,V,94112.0
907,A,94103.0


In [21]:
df['Street Number Suffix'].value_counts()

A    1501
B     291
V     228
C      56
E      28
F      24
G      12
D      11
H      11
K      11
R      10
L      10
J       9
I       7
P       3
N       2
½       1
0       1
Name: Street Number Suffix, dtype: int64

In [20]:
df['Street Number Suffix'].value_counts() / df.shape[0]

A    0.007547
B    0.001463
V    0.001146
C    0.000282
E    0.000141
F    0.000121
G    0.000060
D    0.000055
H    0.000055
K    0.000055
R    0.000050
L    0.000050
J    0.000045
I    0.000035
P    0.000015
N    0.000010
½    0.000005
0    0.000005
Name: Street Number Suffix, dtype: float64

In [43]:
cat_vars_list  = ['Street Suffix', 'Permit Type Definition', 'Street Name','Description','Existing Construction Type Description','Proposed Construction Type Description','Neighborhoods - Analysis Boundaries']
cat_vars  = [i for i in cat_vars_list] 
print("\nCategorical features:\n")
cat_vars
# x.select_dtypes(include=[object]).columns.values.tolist()  

# print("\nCategorical features:\n", cat_vars)


Categorical features:



['Street Suffix',
 'Permit Type Definition',
 'Street Name',
 'Description',
 'Existing Construction Type Description',
 'Proposed Construction Type Description',
 'Neighborhoods - Analysis Boundaries']

In [42]:
num_vars_list  = ['Permit Type', 'Block', 'Street Number', 'Plansets', 'Existing Construction Type','Proposed Construction Type','Zipcode','Supervisor District','Record ID']
num_vars  = [i for i in num_vars_list]
print("\nNumerical features:\n")
num_vars 
# x.select_dtypes(exclude=[object]).columns.values.tolist()

# print("\nNumerical features:\n", num_vars)


Numerical features:



['Permit Type',
 'Block',
 'Street Number',
 'Plansets',
 'Existing Construction Type',
 'Proposed Construction Type',
 'Zipcode',
 'Supervisor District',
 'Record ID']

In [52]:
num_4_treeModels = pipeline.Pipeline(steps=[
   ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False)), # mean, median
])

cat_4_treeModels = pipeline.Pipeline(steps=[
   ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', preprocessing.OrdinalEncoder())
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Permit Type', 'Block', 'Street Number',
                                  'Plansets', 'Existing Construction Type',
                                  'Proposed Construction Type', 'Zipcode',
                                  'Supervisor District', 'Record ID']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder())]),
                                 ['Street Suffix', 'Permit Type Definition',
                                  'Street Name', 'Description',
                                  'Existing Construction Type Description',
                                  'Proposed Construction Type Description',
                                  'Neighborhoods - Analysis Boundaries'])])